![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to run the defect detection pipeline in different playback modes like file and live capture. Live capture has normal mode & demo mode. Normal mode runs at full fps & demo mode is restricted to 4fps. Live images are captured from the MIPI device, then performs defect detection acceleration using Vitis Vision library. The defect results text will be embedded along with the accelerator output image and sends out to 4K display.

Please refer to the tutorial of the SOM Starter Kit Defect Detection document for detailed HW/SW architecture


# 2. Imports and Initialization
Import all python modules required for this notebook.


In [ ]:
import sys
import glob
import subprocess
import re
import os

In [ ]:
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
gi.require_version('GstVideo', '1.0')
gi.require_version('GIRepository', '2.0')
from gi.repository import GObject, Gst, GstVideo, GLib, GIRepository
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

In [ ]:
mainloop = GLib.MainLoop()


## 3. Run the modetest command to configure the mixer IP for 4K

In [ ]:
fwpath="/dev/dri/by-path/"
fwfilename="platform-b0010000.v_mix-card"
if os.path.exists(fwpath) and fwfilename in os.listdir(fwpath):
    out = subprocess.check_output('modetest -M xlnx -D B0010000.v_mix -s 52@40:3840x2160@NV16',shell=True)
    print(out.decode('ascii'))
else:    
    print("Defect detection Firmware not loaded, Run 'xmutil loadapp kv260-defect-detect' ")  
    exit(1)

# 4. Construct the String Representation of GStreamer Pipeline
## The get_media_by_device function returns the matching media node for a given video capture source.<br>
## The following sources are supported in this notebook:

* mipi : platform1 only

In [ ]:
def get_media_dev_by_name(src):
    sources = {
        'mipi' : 'vcap_csi',
    }
    devices = glob.glob('/dev/media*')
    for dev in devices:
        proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
        for line in proc.stdout.splitlines():
            if sources[src] in line:
                return dev

## Change the playback variable to switch between filesrc/media soruce <br>
## Available options:
## 0 ==> File input and file sink
## 1 ==> Live normal mode. Live source and display out normal mode
## 2 ==> Live Demo mode. Live source and display out
## 3 ==> File In Display Out
## 4 ==> Live In and File Out
## 5 ==> File In Display Out - Demo Mode
## Change the filesinkpath to store the different stages output

In [ ]:
# 0 for file input and file sink
# 1 for Live normal mode
# 2 for Live Demo mode
# 3 File In Display Out
# 4 Live In and File Out
# 5 File In Display Out Demo Mode
playback = 1
filesinkpath="/home/petalinux"

## Get the source by calling get_media_dev_by_name()
## For file source copy the input video file to any rootfs path & update the location variable accordingly 

In [ ]:
source = "mipi"

if playback == 1 or playback == 2 or playback == 4:#media src bin
    media_device = get_media_dev_by_name(source)
    print(media_device)
    if media_device is None:
        raise Exception('Unable to find video source ' + source + '. Make sure the device is plugged in, powered and correct Defect detection firmware is loaded.')
    # 'ar0144-sensor-calib.sh' script will set the default sensor calibration paramters
    # User can do the calibration for their test environment & update the values in the script
    os.system('ar0144-sensor-calib.sh '+media_device)

src=""

if playback == 1 or playback == 2 or playback == 4:#media src bin
    src += "mediasrcbin media-device=" + media_device
elif playback == 0 or playback == 3 or playback == 5: # file src
    src += "filesrc location=/home/petalinux/input_video.y8 blocksize=1024000"    

print(src)

## Construct the real pipeline string.

### configuration directory for IVAS plugin

In [ ]:
confdir="/opt/xilinx/share/ivas/defect-detect"

### Set the caps.
User can change the resolution and framerate here.

In case videosrc doesn't support GRAY8 format, adjust the pipeline to fit with followning elements.

In [ ]:
if playback == 0 or playback == 1 or playback == 3 or playback == 4:
    pip=src + ' ! video/x-raw, width=1280, height=800, format=GRAY8, framerate=60/1 '
elif playback == 2:
    pip=src + ' ! video/x-raw, width=1280, height=800, format=GRAY8, framerate=4/1 '
elif playback == 5:    
    pip=src + ' ! rawvideoparse use-sink-caps=false width=1280 height=800 format=gray8 framerate=4/1 '
print(pip)

### Run the live capture pipeline.
ivas_xfilter kconfig="{confdir}/otsu-accelarator.json" is configure the kernel name, xclbin path, debug level. <br>
preprocess-accelarator.json to configure kernel name, xclbin path, threshold value & max value. <br>
cca-accelarator.json is to configure kernel name, xclbin path, debug level. <br>
text2overlay.json is to configure defect threshold, x/y offset of the text, font size, font. <br>

Detailed configuration can be found in the json file.

In [ ]:
pip += ' ! tee name=t_src t_src. ! queue ! ivas_xfilter kernels-config={confdir}/otsu-accelarator.json ! ivas_xfilter kernels-config={confdir}/preprocess-accelarator.json ! tee name=t_pre t_pre. ! queue ! ivas_xfilter kernels-config={confdir}/cca-accelarator.json ! ivas_xfilter kernels-config={confdir}/text2overlay.json '.format(confdir=confdir)

# For demo mode
if playback == 2:
    pip += ' ! videorate ! video/x-raw, width=1280, height=800, format=GRAY8, framerate=4/1 '
    
if playback == 0 or playback == 4:
    pip += "! filesink  location={path}/final.y8".format(path=filesinkpath)
elif playback == 1 or playback == 2 or playback == 3 or playback == 5:
    pip += '! perf ! kmssink bus-id=B0010000.v_mix plane-id=34 render-rectangle="<2560,680,1280,800>"'

pip += ' t_src. ! queue '

# For demo mode
if playback == 2:
    pip += ' ! videorate ! video/x-raw, width=1280, height=800, format=GRAY8, framerate=4/1 '

if playback == 0 or playback == 4:
    pip += "! filesink location={path}/raw_src.y8".format(path=filesinkpath)
elif playback == 1 or playback == 2 or playback == 3 or playback == 5:
    pip += '! perf ! kmssink bus-id=B0010000.v_mix  plane-id=35 render-rectangle="<0,680,1280,800>"'

pip += ' t_pre. ! queue '

# For demo mode
if playback == 2:
    pip += ' ! videorate ! video/x-raw, width=1280, height=800, format=GRAY8, framerate=4/1 '

if playback == 0 or playback == 4:
    pip += "! filesink location={path}/pre-process.y8 async=false ".format(path=filesinkpath)
elif playback == 1 or playback == 2 or playback == 3 or playback == 5:
    pip += '! perf ! kmssink bus-id=B0010000.v_mix plane-id=36 render-rectangle="<1280,680,1280,800>" async=false '.format(confdir=confdir)

print(pip)

In [ ]:
pipe = Gst.parse_launch(pip)
pipe.set_state(Gst.State.PLAYING)

In [ ]:
loop = GLib.MainLoop()
try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    pipe.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 5. Summary
The Jupyter application shows how to:

1. Create a GStreamer pipeline which utilize the IVAS framework to call Vitis Vision Library to do defect detection on the live mango images, and embed the defect result on the final image from accelerator.
3. Live capture is supported

<center>Copyright© 2019 Xilinx</center>